## Sequence GAN from char-rnn
This is a character-level language model using recurrent neural networks based Sequence GAN (SeqGAN).
SeqGAN was proposed to cover discrete sequence data.
In this assignment, you will implement SeqGAN with shakespeare data used in assignment 3.

Original blog post & code:
https://github.com/LantaoYu/SeqGAN

That said, you are allowed to copy paste the codes from the original repo with an additional effort to apply it to our data.
HOWEVER, try to implement the model yourself first, and consider the original source code as a last resort.
You will learn a lot while wrapping around your head during the implementation. And you will understand more clearly in a code level.

### AND MOST IMPORTANTLY, IF YOU JUST BLINDLY COPY PASTE THE CODE, YOU SHALL RUIN YOUR EXAM.
### The exam is designed to be solvable for students that actually have written the code themselves.
At least strictly re-type the codes from the original repo line-by-line, and understand what each line means thoroughly.

## YOU HAVE BEEN WARNED.

Now proceed to the code. You may use textloader in previous assingment or not. You can freely create another python files (\*.py) and then import them. Following codes can be modified as you want. Just make sure that SeqGAN training works.



In [1]:
# ipython magic function for limiting the gpu to be seen for tensorflow
# if you have just 1 GPU, specify the value to 0
# if you have multiple GPUs (nut) and want to specify which GPU to use, specify this value to 0 or 1 or etc.
%env CUDA_DEVICE_ORDER = PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES = 2
# load a bunch of libraries
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.contrib import legacy_seq2seq
import numpy as np
import argparse
import time
import os
from six.moves import cPickle
from six import text_type
import sys

# this module is from the .py file of this folder
# it handles loading texts to digits (aka. tokens) which are recognizable for the model
from utils import TextLoader

# for TensorFlow vram efficiency: if this is not specified, the model hogs all the VRAM even if it's not necessary
# bad & greedy TF! but it has a reason for this design choice FWIW, try googling it if interested
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

data_dir = 'data/tinyshakespeare'
seq_length = 20
batch_size = 32

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [2]:
import os
import pickle
import numpy as np
with open(os.path.join('./data/tinyshakespeare/', 'vocab.pkl'), 'rb') as f:
    vocab = pickle.load(f)
idx_to_vocab = dict()
for i in range(len(vocab)):
    idx_to_vocab[vocab[i]] = i
print(idx_to_vocab)

{' ': 0, 'e': 1, 't': 2, 'o': 3, 'a': 4, 'h': 5, 's': 6, 'r': 7, 'n': 8, 'i': 9, '\n': 10, 'l': 11, 'd': 12, 'u': 13, 'm': 14, 'y': 15, ',': 16, 'w': 17, 'f': 18, 'c': 19, 'g': 20, 'I': 21, 'b': 22, 'p': 23, ':': 24, '.': 25, 'A': 26, 'v': 27, 'k': 28, 'T': 29, "'": 30, 'E': 31, 'O': 32, 'N': 33, 'R': 34, 'S': 35, 'L': 36, 'C': 37, ';': 38, 'W': 39, 'U': 40, 'H': 41, 'M': 42, 'B': 43, '?': 44, 'G': 45, '!': 46, 'D': 47, '-': 48, 'F': 49, 'Y': 50, 'P': 51, 'K': 52, 'V': 53, 'j': 54, 'q': 55, 'x': 56, 'z': 57, 'J': 58, 'Q': 59, 'Z': 60, 'X': 61, '3': 62, '&': 63, '$': 64}


In [4]:
with open(os.path.join('./data/tinyshakespeare/', 'input.txt'), 'rb') as f:
    lines = f.readlines()
    l = list()
    temp = list()
    for line in lines:
        if len(line)==1:
            for i in range(len(temp)):
                temp[i] = list(temp[i].decode('UTF-8'))
                #temp[i] = temp[i][:-1]
                for j in range(len(temp[i])):
                    temp[i][j] = idx_to_vocab[temp[i][j]]
                    
            l.append(sum(temp,[]))
            temp = list()
        else:
            temp.append(line)

print(l[0])

[49, 9, 7, 6, 2, 0, 37, 9, 2, 9, 57, 1, 8, 24, 10, 43, 1, 18, 3, 7, 1, 0, 17, 1, 0, 23, 7, 3, 19, 1, 1, 12, 0, 4, 8, 15, 0, 18, 13, 7, 2, 5, 1, 7, 16, 0, 5, 1, 4, 7, 0, 14, 1, 0, 6, 23, 1, 4, 28, 25, 10]


In [5]:
emb = list()
for _ in range(65):
    emb.append(np.random.normal(0, 0.1, 32))
    
embedding = dict()
for i in range(len(emb)):
    embedding[i] = emb[i]
print(embedding[0])

[ 0.0793435   0.19020333  0.0922966   0.08004362  0.20611638  0.10598402
 -0.02360283  0.0163842   0.13102228 -0.02654421  0.11638848 -0.0094324
  0.00747876 -0.07338662 -0.15279333  0.19423566  0.07700936 -0.03131523
 -0.15540833  0.0645715   0.14683448 -0.00246805 -0.03904026 -0.24221624
  0.04898092  0.09916815 -0.19150893  0.17427377  0.04934009 -0.14824519
  0.02098398 -0.06480274]


In [24]:
l_emb = list()
for i in range(len(l)):
    l_emb.append(list())
    
for i in range(len(l)):
    for j in range(len(l[i])):
        l_emb[i].append(embedding[l[i][j]])

print(l_emb[0][5])

[ 0.0793435   0.19020333  0.0922966   0.08004362  0.20611638  0.10598402
 -0.02360283  0.0163842   0.13102228 -0.02654421  0.11638848 -0.0094324
  0.00747876 -0.07338662 -0.15279333  0.19423566  0.07700936 -0.03131523
 -0.15540833  0.0645715   0.14683448 -0.00246805 -0.03904026 -0.24221624
  0.04898092  0.09916815 -0.19150893  0.17427377  0.04934009 -0.14824519
  0.02098398 -0.06480274]


In [22]:
with open(os.path.join('./save/', 'target_params_py3.pkl'), 'rb') as f:
    target_params = pickle.load(f)
print(len(target_params))
for i in range(len(target_params)):
    print(target_params[i].shape)

15
(5000, 32)
(32, 32)
(32, 32)
(32,)
(32, 32)
(32, 32)
(32,)
(32, 32)
(32, 32)
(32,)
(32, 32)
(32, 32)
(32,)
(32, 5000)
(5000,)
